# Atmospheric Model

It should not come as much surprise that the core of our flight simulation models are derived from a model of the atmosphere that we're flying in. There are many such atmospheric models, but the one used most commonly in aviation is the International Standard Atmosphere.

...


### Symbols used

Since the primary source upon which the models in this package are built from is the ISA, I will use the symbols defined in the paper that defined it.

In [128]:
from symbols import *

display(symbol_table)

Symbol | Description
-------|:-----------
$A$ | Area
$F$ | Force
$g$ | Acceleration due to gravity
$H$ | Geopotential height
$h$ | Geometric height
$P$ | Pressure
$P_{b}$ | Pressure at the base of an atmospheric layer
$M$ | Molecular mass
$m$ | Mass
$n$ | Amount of substance
$R^{*}$ | Universal gas constant
$R$ | Specific gas constant
$T$ | Thermodynamic temperature
$V$ | Volume
$\rho$ | Density

## Atmospheric Model

The basis of all of our calculations and simulations is a model of Earth's atmosphere, of which there are several.  The [International Standard Atmosphere](https://en.wikipedia.org/wiki/International_Standard_Atmosphere) (ISA) is used most commonl in the field of aviation, so that is what we will use.

The ISA is a mathematical model of the atmosphere that assumes a linear temperature gradient against altitude, and derives pressure and density from the [ideal gas law](https://en.wikipedia.org/wiki/Ideal_gas_law) and [hydrostatic equilibrium](https://en.wikipedia.org/wiki/Hydrostatic_equilibrium).

### Hydrostatic equation

Our model begins by noting that since the atmosphere is not floating away into space, nor sinking to the ground, it must be at equilibrium: all vertical forces must sum to zero. That is to say, it is at [hydrostatic equilibrium](https://en.wikipedia.org/wiki/Hydrostatic_equilibrium). For a column of air in the atmosphere, the 3 forces we'll model are from the surrounding air pressure - air pushing up from below, air pushing down from above - and the weight of the column of air itself:

$0 = F_{below} + F_{above} + F_{weight}$

Replacing with the definitions of pressure and weight gives us (For reasons that will become apparent, we will refer to the pressure at the base of the cylinder as $P_b$ and the pressure at the top simply as $P$):

In [70]:
Eq((A*P_b) - (A*P) - (m*g), 0)

Eq(-A*P + A*P_b - g*m, 0)

In [71]:
# The mass of a column of air is its volume multiplied by its density:
# m = V * rho
_.replace(m, V*rho)

Eq(-A*P + A*P_b - V*g*rho, 0)

In [72]:
# The volume is its area multiplied by its height.
_.replace(V, A * h)

Eq(-A*P + A*P_b - A*g*h*rho, 0)

In [73]:
# We can simplify by removing the area:
_ / A

Eq(P - P_b + g*h*rho, 0)

In [74]:
_.replace(P-P_b, diff(P))

Eq(P - P_b + g*h*rho, 0)

In [156]:
z, z_1, z_2, g, R, T_v, dz = symbols('z z_1 z_2 g R T_v dz', real=True)
p_1, p_2, dp, p = symbols('p_1 p_2 dp p', real=True)

P = Symbol('P')
integrate(1/P, (P, p_1, p_2))

-log(p_1) + log(p_2)

### Ideal Gas Law

The first useful equation for us is the [ideal gas law](https://en.wikipedia.org/wiki/Ideal_gas_law), which relates the pressure and volume of an ideal gas to it's temperature. In short: as the temperature of a gas increases, it's pressure and/or volume must also increase.  It is commonly stated as $PV = nR^*T$, but since it can be hard to count the number of molecules in some volume of gas, it is useful to transform into the molar form through some simple substitutions. The molar form is convenient in atmoshpheric science as we can more easily measure air pressure, temperature, and density.

In [8]:
# pV = nRT
ideal_gas_law = Eq(P * V, n * R_universal * T)
ideal_gas_law

Eq(P*V, R^**T*n)

In [9]:
# The amount of substance in a thing is equivalent to its mass / molecular mass:
# n = m / M
ideal_gas_law.subs(n, m / M)

Eq(P*V, R^**T*m/M)

In [10]:
# Density is defined as mass / volume:
# rho = m / V
_.solve(P).subs(m/V, rho)

AttributeError: 'list' object has no attribute 'subs'

In [ ]:
# Finally, the specific gas constant of a gas is defined as the universal gas constant / molecular mass:
# R_sp = R / M
molar_gas_law = _.subs(R_universal/M, R)
molar_gas_law

## Burst Altitude

The maximum diameter of a given balloon is given by its manufacturer.  If we know the volume of gas that we've put into the balloon, we can use the ideal gas law to determine the pressure at which the balloon will burst.



In [ ]:
r = symbols('r')

volume_of_sphere = Eq(V, Rational(4,3)*pi*r**3)
volume_of_sphere

foo = ideal_gas_law.subs(volume_of_sphere.lhs, volume_of_sphere.rhs)
display(foo)

solved = next(iter(solveset(foo, P)))

from sympy.printing.pycode import pycode

display(solved)
pycode(solved)

In [ ]:
from sympy import *

F_b, m, g = symbols('F_b m g')
rho, V, r = symbols('rho V r')
rho_air, rho_gas = symbols('rho_air rho_gas')

bouyancy = Eq(F_b, m*g)
bouyancy = bouyancy.subs({
    m: rho*V,
    rho: (rho_gas - rho_air),
    V: (Rational(4,3)*pi*r**3)
})

bouyancy

## References

* [ISO 2533-1975: Standard Atmosphere](https://cdn.standards.iteh.ai/samples/7472/c203e9121d4c40e5bdc98844b1a1e2f4/ISO-2533-1975.pdf)
* [Inspiration](https://github.com/cuspaceflight/cusf-burst-calc/blob/master/js/calc.js)
* [And](https://northstar-www.dartmouth.edu/~klynch/pmwiki-gc/uploads/BalloonCalulations.pdf)
* [And](https://amt.copernicus.org/articles/4/2235/2011/amt-4-2235-2011.pdf)
